<a href="https://colab.research.google.com/github/uamcode/AIDAM_final/blob/master/quality_evaluation/src/models%20/DE_RoBERTa_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 패키지 호출

In [ ]:
!pip install scikit-learn wandb bitsandbytes peft accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 31.1 MB/s eta 0:00:00


In [ ]:
import glob
import numpy as np
import pandas as pd
import os
import json
import zipfile

from datasets import Dataset
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import (
    DebertaV2ForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    EvalPrediction
)

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

from sklearn.model_selection import GroupShuffleSplit
from IPython.display import display


In [ ]:
import os
from google.colab import userdata

# Hugging Face
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
os.environ["HUGGINGFACE_HUB_TOKEN"] = os.environ["HF_TOKEN"]  # 안전빵

# Weights & Biases
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 모델 구조 파악하기

In [ ]:
# klue roberta
from transformers import AutoModel, AutoTokenizer

model1 = AutoModel.from_pretrained("klue/roberta-base")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
print(model1)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [ ]:
# DeBERTa v3 모델
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("team-lucid/deberta-v3-base-korean")
model = DebertaV2ForSequenceClassification.from_pretrained("team-lucid/deberta-v3-base-korean")

inputs = tokenizer("안녕, 세상!", return_tensors="pt")
outputs = model(**inputs)

tokenizer_config.json:   0%|          | 0.00/390 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at team-lucid/deberta-v3-base-korean and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(64100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerN

## 데이터 불러오기

In [ ]:
# 허깅페이스 로그인 확인
from huggingface_hub import whoami
print(whoami())

{'type': 'user', 'id': '672d65f605fbc0acf9e7c359', 'name': 'yj512', 'fullname': 'syj', 'email': 'tladbwls70@naver.com', 'emailVerified': True, 'canPay': False, 'billingMode': 'prepaid', 'periodEnd': 1769904000, 'isPro': False, 'avatarUrl': '/avatars/1e14179ec4beadc1d14bec0dd5c8ae21.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'uj_LLM', 'role': 'write', 'createdAt': '2025-06-05T16:38:05.860Z'}}}


In [ ]:
# zip 파일 해제하기
path='/content/drive/MyDrive/멋쟁이사자처럼/project1_data/'
folder1='TL_TL_발화·대화단위 평가 데이터_TL_경제활동_상품상거래.zip'
folder2='TL_TL_발화·대화단위 평가 데이터_TL_기술_과학.zip'
folder3='TL_TL_발화·대화단위 평가 데이터_TL_미용_건강_식음료.zip'
folder4='TL_TL_발화·대화단위 평가 데이터_TL_엔터테인먼트_오락_예술.zip'
folder5='TL_TL_발화·대화단위 평가 데이터_TL_여행_여가_취미.zip'
folder6='TL_TL_발화·대화단위 평가 데이터_TL_인문사회.zip'
folder7='TL_TL_발화·대화단위 평가 데이터_TL_주거_생활_사람관계.zip'

# 전체 경로 생성
path1 = path + folder1
path2 = path + folder2
path3 = path + folder3
path4 = path + folder4
path5 = path + folder5
path6 = path + folder6
path7 = path + folder7

# 압축 해제
zip_list = [path1, path2, path3, path4, path5, path6, path7]

for zip_path in zip_list:
    print(f"Extracting: {zip_path}")
    try:
        # .zip을 뗀 폴더 이름 생성
        extract_folder = os.path.splitext(zip_path)[0]

        # 폴더 생성 (이미 존재하면 무시)
        os.makedirs(extract_folder, exist_ok=True)

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
        print(f"Done. Extracted to {extract_folder}")
    except Exception as e:
        print(f"Error extracting {zip_path}: {e}")

Extracting: /content/drive/MyDrive/멋쟁이사자처럼/project1_data/TL_TL_발화·대화단위 평가 데이터_TL_경제활동_상품상거래.zip
Done. Extracted to /content/drive/MyDrive/멋쟁이사자처럼/project1_data/TL_TL_발화·대화단위 평가 데이터_TL_경제활동_상품상거래
Extracting: /content/drive/MyDrive/멋쟁이사자처럼/project1_data/TL_TL_발화·대화단위 평가 데이터_TL_기술_과학.zip
Done. Extracted to /content/drive/MyDrive/멋쟁이사자처럼/project1_data/TL_TL_발화·대화단위 평가 데이터_TL_기술_과학
Extracting: /content/drive/MyDrive/멋쟁이사자처럼/project1_data/TL_TL_발화·대화단위 평가 데이터_TL_미용_건강_식음료.zip
Done. Extracted to /content/drive/MyDrive/멋쟁이사자처럼/project1_data/TL_TL_발화·대화단위 평가 데이터_TL_미용_건강_식음료
Extracting: /content/drive/MyDrive/멋쟁이사자처럼/project1_data/TL_TL_발화·대화단위 평가 데이터_TL_엔터테인먼트_오락_예술.zip
Done. Extracted to /content/drive/MyDrive/멋쟁이사자처럼/project1_data/TL_TL_발화·대화단위 평가 데이터_TL_엔터테인먼트_오락_예술
Extracting: /content

## 데이터 전처리

In [ ]:
# train test 분리 버전
def load_csv_data(directory):
    all_files_in_dir = glob.glob(os.path.join(directory, "*.csv"))
    df_list = []
    print(f"Loading CSV files from: {directory}")

    if not all_files_in_dir:
        print(f"No CSV files found in {directory}. Returning empty DataFrame.")
        return pd.DataFrame()

    for file_path in all_files_in_dir:
        filename = os.path.basename(file_path)
        print(f"Attempting to load: {file_path}")
        current_df = None
        try:
            current_df = pd.read_csv(file_path, encoding="utf-8")
            print(f"Successfully loaded {filename} with utf-8 encoding.")
        except UnicodeDecodeError:
            try:
                current_df = pd.read_csv(file_path, encoding="cp949")
                print(f"Successfully loaded {filename} with cp949 encoding.")
            except UnicodeDecodeError:
                try:
                    current_df = pd.read_csv(file_path, encoding="euc-kr")
                    print(f"Successfully loaded {filename} with euc-kr encoding.")
                except pd.errors.EmptyDataError:
                    print(f"Skipping empty CSV file: {filename}")
                except FileNotFoundError:
                    print(f"File not found: {filename}")
                except Exception as e:
                    print(f"Error loading {filename} with euc-kr encoding: {e}")
            except pd.errors.EmptyDataError:
                print(f"Skipping empty CSV file: {filename}")
            except FileNotFoundError:
                print(f"File not found: {filename}")
            except Exception as e:
                print(f"An unexpected error occurred while loading {filename} with cp949 encoding: {e}")
        except pd.errors.EmptyDataError:
            print(f"Skipping empty CSV file: {filename}")
        except FileNotFoundError:
            print(f"File not found: {filename}")
        except Exception as e:
            print(f"An unexpected error occurred while loading {filename} with utf-8 encoding: {e}")

        if current_df is not None:
            df_list.append(current_df)

    if df_list:
        return pd.concat(df_list, ignore_index=True)
    else:
        print(f"No dataframes were successfully loaded from {directory}. Returning empty DataFrame.")
        return pd.DataFrame()


# =========================
# 1) Load Train / Val CSVs
# =========================
print("Loading Train Data...")
train_df = load_csv_data("/content/drive/MyDrive/멋쟁이사자처럼/data/csv_train_data")

print("Loading Validation Data...")
val_df = load_csv_data("/content/drive/MyDrive/멋쟁이사자처럼/data/csv_val_data")

print(f"Raw Train size: {len(train_df)}, Raw Val size: {len(val_df)}")


# =========================
# 2) Label setup
# =========================
labels = [
    "linguistic_acceptability", "consistency", "interestingness", "unbias",
    "harmlessness", "no_hallucination", "understandability", "sensibleness", "specificity"
]

id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}

path_label = "/content/drive/MyDrive/멋쟁이사자처럼/data/"
with open(os.path.join(path_label, "label2id.json"), "w", encoding="utf-8") as f:
    json.dump(label2id, f, ensure_ascii=False)

print("✅ label2id.json saved.")


# =========================
# 3) Voting consolidation
# =========================
def consolidate_labels_by_voting(df: pd.DataFrame) -> pd.DataFrame:
    label_to_numeric = {"yes": 1, "no": 0}

    df_numeric = df.copy()
    for label_col in labels:
        df_numeric[label_col] = df_numeric[label_col].map(label_to_numeric)

    aggregation_dict = {"text": "first"}
    for label_col in labels:
        aggregation_dict[label_col] = "sum"

    grouped_df = (
        df_numeric
        .groupby(["conversation_id", "utterance_id"])
        .agg(aggregation_dict)
        .reset_index()
    )

    # voting rule: 2명 이상 yes면 yes (예: 3명 중 2명 이상)
    for label_col in labels:
        grouped_df[label_col] = grouped_df[label_col].apply(lambda x: "yes" if x >= 2 else "no")

    return grouped_df


print("Consolidating train_df...")
consolidated_train_df = consolidate_labels_by_voting(train_df)

print("Consolidating val_df...")
consolidated_val_df = consolidate_labels_by_voting(val_df)

print("Consolidated Train shape:", consolidated_train_df.shape)
print("Consolidated Val shape  :", consolidated_val_df.shape)


# =========================
# 4) Create conversation history (context)
# =========================
def create_conversation_history(df: pd.DataFrame, keep_last_n=5, sep_token=" [SEP] ") -> pd.DataFrame:
    df_with_history = df.copy()

    # utterance_id에서 숫자 추출 (예: "u3" -> 3)
    df_with_history["utterance_num"] = df_with_history["utterance_id"].apply(lambda x: int(str(x).split("u")[1]))
    df_with_history = df_with_history.sort_values(by=["conversation_id", "utterance_num"]).reset_index(drop=True)

    df_with_history["context"] = ""

    for conv_id, group in df_with_history.groupby("conversation_id"):
        history_list = []
        for idx in group.index:
            recent_history = history_list[-keep_last_n:]
            df_with_history.loc[idx, "context"] = sep_token.join(recent_history)
            history_list.append(df_with_history.loc[idx, "text"])

    df_with_history = df_with_history.drop(columns=["utterance_num"])
    return df_with_history


print("Creating conversation history for consolidated_train_df...")
train_df_with_history = create_conversation_history(consolidated_train_df, keep_last_n=5)

print("Creating conversation history for consolidated_val_df...")
val_df_with_history = create_conversation_history(consolidated_val_df, keep_last_n=5)

print("Train with history head:")
display(train_df_with_history.head())

print("Val with history head:")
display(val_df_with_history.head())


# =========================
# 5) Train/Test split (conversation_id 기준, 누수 방지)
# =========================
def split_train_test_by_conversation(df: pd.DataFrame, test_size=0.1, seed=42):
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
    groups = df["conversation_id"].values

    train_idx, test_idx = next(gss.split(df, groups=groups))
    train_split = df.iloc[train_idx].reset_index(drop=True)
    test_split = df.iloc[test_idx].reset_index(drop=True)
    return train_split, test_split


print("Splitting train_df_with_history into train/test (by conversation_id)...")
train_split_df, test_split_df = split_train_test_by_conversation(
    train_df_with_history,
    test_size=0.1,   # 원하는 test 비율로 변경 가능
    seed=42
)

print("==== Split sizes ====")
print("Train:", len(train_split_df))
print("Test :", len(test_split_df))
print("Val  :", len(val_df_with_history))

# (선택) 대화 ID 겹침 확인
overlap = set(train_split_df["conversation_id"]) & set(test_split_df["conversation_id"])
print("conversation_id overlap:", len(overlap))

print("Train split head:")
display(train_split_df.head())

print("Test split head:")
display(test_split_df.head())

Loading Train Data...
Loading CSV files from: /content/drive/MyDrive/멋쟁이사자처럼/data/csv_train_data
Attempting to load: /content/drive/MyDrive/멋쟁이사자처럼/data/csv_train_data/TL_TL_발화·대화단위 평가 데이터_TL_기술_과학.csv
Successfully loaded TL_TL_발화·대화단위 평가 데이터_TL_기술_과학.csv with utf-8 encoding.
Attempting to load: /content/drive/MyDrive/멋쟁이사자처럼/data/csv_train_data/TL_TL_발화·대화단위 평가 데이터_TL_미용_건강_식음료.csv
Successfully loaded TL_TL_발화·대화단위 평가 데이터_TL_미용_건강_식음료.csv with utf-8 encoding.
Attempting to load: /content/drive/MyDrive/멋쟁이사자처럼/data/csv_train_data/TL_TL_발화·대화단위 평가 데이터_TL_경제활동_상품상거래.csv
Successfully loaded TL_TL_발화·대화단위 평가 데이터_TL_경제활동_상품상거래.csv with utf-8 encoding.
Attempting to load: /content/drive/MyDrive/멋쟁이사자처럼/data/csv_train_data/TL_TL_발화·대화단위 평가 데이터_TL_엔터테인먼트_오락_예술.csv
Successfully loaded TL_TL_발화·대화단위 평가 데이터

,conversation_id,utterance_id,text,linguistic_acceptability,consistency,interestingness,unbias,harmlessness,no_hallucination,understandability,sensibleness,specificity,context
0,1,c1.u2,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...,no,yes,yes,yes,yes,yes,no,yes,yes,
1,1,c1.u4,"금서읽기주간은 2015년에 시작된 행사로, 보수단체의 도서 검열 운동에 대항하기 위...",no,yes,yes,yes,yes,yes,yes,yes,yes,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...
2,1,c1.u6,금서읽기주간에서는 과거에 금서로 지정되었던 책들을 읽고 토론하는 것이 주요 활동입니...,no,yes,yes,yes,yes,yes,yes,yes,yes,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...
3,1,c1.u8,"과거에는 권력자들이 통제를 위해 금서를 지정했는데, 예를 들어 젊은 베르테르의 슬픔...",no,yes,yes,yes,yes,yes,yes,yes,yes,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...
4,1,c1.u10,"네, 최근에도 도서관에서 성교육, 성평등 도서를 빼라는 요구가 있었고, 이에 대한 ...",no,yes,yes,yes,yes,yes,yes,yes,yes,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...


Val with history head:


,conversation_id,utterance_id,text,linguistic_acceptability,consistency,interestingness,unbias,harmlessness,no_hallucination,understandability,sensibleness,specificity,context
0,5,c5.u2,"네, 최헌의 앵두는 2023년에 젊은이들 사이에서 인기를 끌었어요. 이 곡은 그 당...",yes,yes,yes,yes,yes,yes,yes,yes,yes,
1,5,c5.u4,"네, 그렇게 봐야죠.",yes,yes,yes,yes,yes,yes,yes,yes,yes,"네, 최헌의 앵두는 2023년에 젊은이들 사이에서 인기를 끌었어요. 이 곡은 그 당..."
2,5,c5.u6,"'밀수'는 여러 옛 노래들이 사용되었어요. 예를 들면 '잘 살아보세', '연안부두'...",yes,yes,yes,yes,yes,yes,yes,yes,yes,"네, 최헌의 앵두는 2023년에 젊은이들 사이에서 인기를 끌었어요. 이 곡은 그 당..."
3,5,c5.u8,'밀수'에서 사용된 옛 노래들은 극적인 효과를 주었어요. 이 노래들은 각 장면과 인...,yes,yes,yes,yes,yes,yes,yes,yes,yes,"네, 최헌의 앵두는 2023년에 젊은이들 사이에서 인기를 끌었어요. 이 곡은 그 당..."
4,5,c5.u10,'마스크걸'에서는 김모미의 내면을 대변하는 옛 음악들이 사용되었어요. 예를 들면 김...,yes,yes,yes,yes,yes,yes,yes,yes,yes,"네, 최헌의 앵두는 2023년에 젊은이들 사이에서 인기를 끌었어요. 이 곡은 그 당..."


Splitting train_df_with_history into train/test (by conversation_id)...
==== Split sizes ====
Train: 360530
Test : 40042
Val  : 50047
conversation_id overlap: 0
Train split head:


,conversation_id,utterance_id,text,linguistic_acceptability,consistency,interestingness,unbias,harmlessness,no_hallucination,understandability,sensibleness,specificity,context
0,1,c1.u2,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...,no,yes,yes,yes,yes,yes,no,yes,yes,
1,1,c1.u4,"금서읽기주간은 2015년에 시작된 행사로, 보수단체의 도서 검열 운동에 대항하기 위...",no,yes,yes,yes,yes,yes,yes,yes,yes,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...
2,1,c1.u6,금서읽기주간에서는 과거에 금서로 지정되었던 책들을 읽고 토론하는 것이 주요 활동입니...,no,yes,yes,yes,yes,yes,yes,yes,yes,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...
3,1,c1.u8,"과거에는 권력자들이 통제를 위해 금서를 지정했는데, 예를 들어 젊은 베르테르의 슬픔...",no,yes,yes,yes,yes,yes,yes,yes,yes,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...
4,1,c1.u10,"네, 최근에도 도서관에서 성교육, 성평등 도서를 빼라는 요구가 있었고, 이에 대한 ...",no,yes,yes,yes,yes,yes,yes,yes,yes,독서문화시민연대는 독서를 통해 시민들이 생각하고 판단하며 성찰하는 시민이 되기 위해...


Test split head:


,conversation_id,utterance_id,text,linguistic_acceptability,consistency,interestingness,unbias,harmlessness,no_hallucination,understandability,sensibleness,specificity,context
0,6,c6.u2,"네, 양진석은 건축가이자 싱어송라이터로 활동하고 있는 분입니다. 그는 2000년대 ...",yes,yes,yes,yes,yes,yes,yes,yes,yes,
1,6,c6.u4,그렇습니다. 정말로 그렇죠.,yes,yes,no,yes,yes,yes,yes,no,no,"네, 양진석은 건축가이자 싱어송라이터로 활동하고 있는 분입니다. 그는 2000년대 ..."
2,6,c6.u6,양진석은 음악과 건축을 조화시키기 위해 노력하고 있습니다. 건축 일이 몰리면서 음악...,yes,yes,yes,yes,yes,yes,yes,yes,yes,"네, 양진석은 건축가이자 싱어송라이터로 활동하고 있는 분입니다. 그는 2000년대 ..."
3,6,c6.u8,7집 '프리덤'은 양진석이 12년 만에 싱어송라이터로 돌아온 앨범입니다. 이번 앨범...,yes,yes,yes,yes,yes,yes,yes,yes,yes,"네, 양진석은 건축가이자 싱어송라이터로 활동하고 있는 분입니다. 그는 2000년대 ..."
4,6,c6.u10,"이번 앨범에는 경쾌하고 청량한 시티팝, 일렉트로닉 팝, 록 스타일의 곡들이 수록되어...",yes,yes,yes,yes,yes,yes,yes,yes,yes,"네, 양진석은 건축가이자 싱어송라이터로 활동하고 있는 분입니다. 그는 2000년대 ..."


## 모델 학습하기

In [ ]:
labels = ['linguistic_acceptability', 'consistency', 'interestingness', 'unbias',
          'harmlessness', 'no_hallucination', 'understandability', 'sensibleness', 'specificity']

id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}

In [ ]:
import wandb

# 로깅
wandb.init(
    project="DeBERTa_log",
    name='DeBERTa_lora2',
    config={
        # 실험 정체성
        "task": "dialog_turn_multilabel_classification",
        "model": "team-lucid/deberta-v3-base-korean",
        "tokenizer": "team-lucid/deberta-v3-base-korean",

        # 데이터 / 입력 설계
        "input_format": "bot_answer | history",
        "history_scope": "until_previous_turn",
        "max_seq_len": 512,

        # 라벨 설정
        "label_type": "multi_label",
        "num_labels": len(labels),          # 라벨 수
        "label_names": labels,              # ["correct", "polite", ...]

        # 학습 하이퍼파라미터
        "batch_size": 16,
        "valid_batch_size": 16,
        "gradient_accumulation": 1,
        "epochs": 3,
        "learning_rate": 2e-5,
        "weight_decay": 0.01,

        # 평가 설정
        "metric": "f1_micro",
        "threshold": 0.5,

        # 재현성
        "seed": 42,
    }
)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: uam_study70 (uam_study70-hanseo-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("team-lucid/deberta-v3-base-korean")

def preprocess_function(examples):
    # Segment A: text (평가 대상 발화)
    # Segment B: context (그전 대화 맥락)
    # truncation="only_second"는 전체 길이가 길어질 때
    # 평가 대상인 Segment A는 보존하고 맥락인 Segment B의 앞쪽을 자르라는 의미입니다.

    # 'text'나 'context'가 None(NaN)인 경우를 대비해 빈 문자열 처리
    texts = [str(t) if t is not None else "" for t in examples["text"]]
    contexts = [str(c) if c is not None else "" for c in examples["context"]]

    # 두 문장을 결합하여 토큰화 (truncation_side='left'에 의해 앞의 context부터 잘림)
    encoding = tokenizer(
        contexts, # 첫 번째 문장 (맥락)
        texts,    # 두 번째 문장 (현재 발화)
        padding="max_length",
        truncation=True, # 전체 순서에서 왼쪽부터 자름
        max_length=512
    )
    # 레이블 처리 (기존과 동일)
    labels_matrix = []
    for i in range(len(examples["text"])):
        label_vector = [1.0 if examples[label][i] == 'yes' else 0.0 for label in labels]
        labels_matrix.append(label_vector)
    encoding["labels"] = labels_matrix
    return encoding

# Convert to HuggingFace Dataset, using the new DataFrames with history
train_dataset = Dataset.from_pandas(train_df_with_history)
val_dataset = Dataset.from_pandas(val_df_with_history)
test_dataset  = Dataset.from_pandas(test_split_df)

print("Tokenizing Datasets with conversation history...")
encoded_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
encoded_val = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)
encoded_test = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

tokenizer_config.json:   0%|          | 0.00/390 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
encoded_test

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 40042
})

In [ ]:
# 데이터 드라이브에 저장
encoded_train.save_to_disk("/content/drive/MyDrive/멋쟁이사자처럼/data/encoded_train")
encoded_val.save_to_disk("/content/drive/MyDrive/멋쟁이사자처럼/data/encoded_val")
encoded_test.save_to_disk("/content/drive/MyDrive/멋쟁이사자처럼/data/encoded_test")

Saving the dataset (0/3 shards):   0%|          | 0/400572 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50047 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/40042 [00:00<?, ? examples/s]

In [ ]:
# 데이터 호출하기
from datasets import load_from_disk
encoded_test = load_from_disk("/content/drive/MyDrive/멋쟁이사자처럼/data/encoded_test")
encoded_val = load_from_disk("/content/drive/MyDrive/멋쟁이사자처럼/data/encoded_val")
encoded_train=load_from_disk("/content/drive/MyDrive/멋쟁이사자처럼/data/encoded_train")

## 풀파인튜닝

In [ ]:
# 모델 호출하기
model = DebertaV2ForSequenceClassification.from_pretrained(
    "team-lucid/deberta-v3-base-korean",
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id)

model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(
    output_dir="your-model",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    push_to_hub=True,
    hub_model_id="uamcode/deberta-korean-multilabel_base",
    report_to=["wandb"]
)

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average='micro')
    # accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
                'roc_auc': roc_auc}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds, labels=p.label_ids)
    return result

trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at team-lucid/deberta-v3-base-korean and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2932192011.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## LoRA

In [ ]:
# Q_LoRA 한번 써볼까 후 - DeBERTa 모델은 Q-lora 불가능 그냥 lora만 적용 > 8-bit 양자화 설정 > 이것도 모델 구조 때문에 안됨
# 그래서 그냥 LoRA만 적용함

# 1) Base 모델/토크나이저 로드
# =========================
BASE_MODEL_ID = "team-lucid/deberta-v3-base-korean"
tokenizer = AutoTokenizer.from_pretrained("team-lucid/deberta-v3-base-korean")

model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL_ID,
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

# tokenizer에 special token을 "추가"한 경우에만 사용
# model.resize_token_embeddings(len(tokenizer))


# =========================
# 2) LoRA 설정 + 결합
# =========================
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=["query_proj", "key_proj", "value_proj", "dense"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


# =========================
# 3) Metrics (멀티라벨)
# =========================
def multi_label_metrics(predictions, labels, threshold=0.5):
    probs = torch.sigmoid(torch.tensor(predictions))
    y_pred = (probs.numpy() >= threshold).astype(int)

    f1_micro = f1_score(y_true=labels, y_pred=y_pred, average="micro")
    roc_auc = roc_auc_score(labels, y_pred, average="micro")
    return {"f1": f1_micro, "roc_auc": roc_auc}

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    return multi_label_metrics(predictions=preds, labels=p.label_ids)


# =========================
# 4) TrainingArguments
# =========================
training_args = TrainingArguments(
    output_dir="your-model-lora2",
    learning_rate=2e-4,                 # LoRA는 보통 LR 크게
    per_device_train_batch_size=96,     # A100 80GB면 더 키워도 됨(예: 128)
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    bf16=True,                          # A100이면 bf16 추천
    report_to=["wandb"],
    push_to_hub=True,
    hub_model_id="yj512/deberta-korean-multilabel_base-lora2",
)


# =========================
# 5) Trainer 학습
# =========================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

# best adapter 저장
trainer.save_model("your-model-lora2/best")
tokenizer.save_pretrained("your-model-lora2/best")

# hub에 한 번 더 안전빵 push
trainer.push_to_hub()

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at team-lucid/deberta-v3-base-korean and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

  ...l-lora2/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...adapter_model.safetensors: 100%|##########| 10.8MB / 10.8MB            

  ...adapter_model.safetensors: 100%|##########| 10.8MB / 10.8MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/yj512/deberta-korean-multilabel_base-lora2/commit/fa0b6baf776b511f26240ba6b58eef718e9591ec', commit_message='End of training', commit_description='', oid='fa0b6baf776b511f26240ba6b58eef718e9591ec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yj512/deberta-korean-multilabel_base-lora2', endpoint='https://huggingface.co', repo_type='model', repo_id='yj512/deberta-korean-multilabel_base-lora2'), pr_revision=None, pr_num=None)